# WE'LL START WITH WEATHER DATA

In [5]:
import requests
from dateutil import parser, rrule
from datetime import datetime, time, date
import time
from preprocessor import *


In [6]:
station = 'IEDINBUR6' # Edinburgh
data_raw = pd.read_csv('~/Documents/datasets/weather/{}_weather.csv'.format(station),index_col=0)
prep = Preprocessor()
prep.overview(data_raw)

Dataset contains 101443 rows and 17 columns
Data types of the raw (uncleaned) data:
Time                        object
TemperatureC               float64
DewpointC                  float64
PressurehPa                float64
WindDirection               object
WindDirectionDegrees         int64
WindSpeedKMH               float64
WindSpeedGustKMH           float64
Humidity                     int64
HourlyPrecipMM             float64
Conditions                 float64
Clouds                     float64
dailyrainMM                float64
SolarRadiationWatts/m^2    float64
SoftwareType                object
DateUTC                     object
station                     object
dtype: object
Categorical features of the dataset: ['Time', 'WindDirection', 'SoftwareType', 'DateUTC', 'station']
Numeric features of the dataset: ['TemperatureC', 'DewpointC', 'PressurehPa', 'WindDirectionDegrees', 'WindSpeedKMH', 'WindSpeedGustKMH', 'Humidity', 'HourlyPrecipMM', 'Conditions', 'Clouds', 'dailyrainMM',

In [7]:
# Sample look at the raw data
data_raw.head()


,Time,TemperatureC,DewpointC,PressurehPa,WindDirection,WindDirectionDegrees,WindSpeedKMH,WindSpeedGustKMH,Humidity,HourlyPrecipMM,Conditions,Clouds,dailyrainMM,SolarRadiationWatts/m^2,SoftwareType,DateUTC,station
0,2015-01-01 00:00:00,10.3,4.5,1001.9,SW,236,9.0,11.9,67,0.0,NaN,NaN,0.0,0.0,HP1001 V1.1.3,2015-01-01 00:00:00,IEDINBUR6
1,2015-01-01 00:06:00,10.2,4.4,1001.9,SW,226,6.9,7.9,67,0.0,NaN,NaN,0.0,0.0,HP1001 V1.1.3,2015-01-01 00:06:00,IEDINBUR6
2,2015-01-01 00:11:00,10.2,4.4,1001.9,SSW,202,7.2,15.8,67,0.0,NaN,NaN,0.0,0.0,HP1001 V1.1.3,2015-01-01 00:11:00,IEDINBUR6
3,2015-01-01 00:16:00,10.2,4.4,1001.9,SSW,202,7.9,11.9,67,0.0,NaN,NaN,0.0,0.0,HP1001 V1.1.3,2015-01-01 00:16:00,IEDINBUR6
4,2015-01-01 00:21:00,10.2,4.6,1002.3,South,174,4.3,7.9,68,0.0,NaN,NaN,0.0,0.0,HP1001 V1.1.3,2015-01-01 00:21:00,IEDINBUR6


In [4]:
# Give the variables some friendlier names and convert types as necessary.
data_raw['temp'] = data_raw['TemperatureC'].astype(float)
data_raw['rain'] = data_raw['HourlyPrecipMM'].astype(float)
data_raw['total_rain'] = data_raw['dailyrainMM'].astype(float)
data_raw['date'] = data_raw['DateUTC'].apply(parser.parse)
data_raw['humidity'] = data_raw['Humidity'].astype(float)
data_raw['wind_direction'] = data_raw['WindDirectionDegrees']
data_raw['wind'] = data_raw['WindSpeedKMH']


In [5]:
data_raw.head()

,Time,TemperatureC,DewpointC,PressurehPa,WindDirection,WindDirectionDegrees,WindSpeedKMH,WindSpeedGustKMH,Humidity,HourlyPrecipMM,...,SoftwareType,DateUTC,station,temp,rain,total_rain,date,humidity,wind_direction,wind
0,2015-01-01 00:00:00,10.3,4.5,1001.9,SW,236,9.0,11.9,67,0.0,...,HP1001 V1.1.3,2015-01-01 00:00:00,IEDINBUR6,10.3,0.0,0.0,2015-01-01 00:00:00,67.0,236,9.0
1,2015-01-01 00:06:00,10.2,4.4,1001.9,SW,226,6.9,7.9,67,0.0,...,HP1001 V1.1.3,2015-01-01 00:06:00,IEDINBUR6,10.2,0.0,0.0,2015-01-01 00:06:00,67.0,226,6.9
2,2015-01-01 00:11:00,10.2,4.4,1001.9,SSW,202,7.2,15.8,67,0.0,...,HP1001 V1.1.3,2015-01-01 00:11:00,IEDINBUR6,10.2,0.0,0.0,2015-01-01 00:11:00,67.0,202,7.2
3,2015-01-01 00:16:00,10.2,4.4,1001.9,SSW,202,7.9,11.9,67,0.0,...,HP1001 V1.1.3,2015-01-01 00:16:00,IEDINBUR6,10.2,0.0,0.0,2015-01-01 00:16:00,67.0,202,7.9
4,2015-01-01 00:21:00,10.2,4.6,1002.3,South,174,4.3,7.9,68,0.0,...,HP1001 V1.1.3,2015-01-01 00:21:00,IEDINBUR6,10.2,0.0,0.0,2015-01-01 00:21:00,68.0,174,4.3


In [6]:
# Extract out only the data we need.
data = data_raw.loc[:, ['date', 'station', 'temp', 'rain', 'total_rain', 'humidity', 'wind']]
data = data[(data['date'] >= datetime(2015,1,1)) & (data['date'] <= datetime(2015,12,31))]
data.head()

,date,station,temp,rain,total_rain,humidity,wind
0,2015-01-01 00:00:00,IEDINBUR6,10.3,0.0,0.0,67.0,9.0
1,2015-01-01 00:06:00,IEDINBUR6,10.2,0.0,0.0,67.0,6.9
2,2015-01-01 00:11:00,IEDINBUR6,10.2,0.0,0.0,67.0,7.2
3,2015-01-01 00:16:00,IEDINBUR6,10.2,0.0,0.0,67.0,7.9
4,2015-01-01 00:21:00,IEDINBUR6,10.2,0.0,0.0,68.0,4.3


In [7]:
# There's an issue with some stations that record rainfall ~-2500 where data is missing.
if (data['rain'] < -500).sum() > 10:
    print("There's more than 10 messed up days for {}".format(station))
    
# remove the bad samples
data = data[data['rain'] > -500]

In [8]:
# Assign the "day" to every date entry
data['day'] = data['date'].apply(lambda x: x.date())

# Get the time, day, and hour of each timestamp in the dataset
data['time_of_day'] = data['date'].apply(lambda x: x.time())
data['day_of_week'] = data['date'].apply(lambda x: x.weekday())    
data['hour_of_day'] = data['time_of_day'].apply(lambda x: x.hour)
# Mark the month for each entry so we can look at monthly patterns
data['month'] = data['date'].apply(lambda x: x.month)


,date,station,temp,rain,total_rain,humidity,wind,day,time_of_day,day_of_week,hour_of_day,month
0,2015-01-01 00:00:00,IEDINBUR6,10.3,0.0,0.0,67.0,9.0,2015-01-01,00:00:00,3,0,1
1,2015-01-01 00:06:00,IEDINBUR6,10.2,0.0,0.0,67.0,6.9,2015-01-01,00:06:00,3,0,1
2,2015-01-01 00:11:00,IEDINBUR6,10.2,0.0,0.0,67.0,7.2,2015-01-01,00:11:00,3,0,1
3,2015-01-01 00:16:00,IEDINBUR6,10.2,0.0,0.0,67.0,7.9,2015-01-01,00:16:00,3,0,1
4,2015-01-01 00:21:00,IEDINBUR6,10.2,0.0,0.0,68.0,4.3,2015-01-01,00:21:00,3,0,1


In [33]:
# Is each time stamp on a working day (Mon-Fri)
data['working_day'] = (data['day_of_week'] >= 0) & (data['day_of_week'] <= 4)

# Classify into morning or evening times (assuming travel between 8.15-9am and 5.15-6pm)
data['morning'] = (data['hour_of_day'] >= 7) & (data['hour_of_day'] <= 9)
data['evening'] = (data['hour_of_day'] >= 17) & (data['hour_of_day'] <= 18)

# If there's any rain at all, mark that!
data['raining'] = data['rain'] > 0.0

# You get wet cycling if its a working day, and its raining at the travel times!
data['get_wet_cycling'] = (data['working_day']) & ((data['morning'] & data['rain']) |
                                                   (data['evening'] & data['rain']))


In [37]:
data.loc[(data['get_wet_cycling'] == True) ].head()

,date,station,temp,rain,total_rain,humidity,wind,day,time_of_day,day_of_week,hour_of_day,month,working_day,raining,morning,evening,get_wet_cycling
91,2015-01-01 08:00:00,IEDINBUR6,10.4,1.8,0.3,80.0,6.4,2015-01-01,08:00:00,3,8,1,True,True,True,False,True
92,2015-01-01 08:05:00,IEDINBUR6,10.3,1.8,0.3,82.0,6.9,2015-01-01,08:05:00,3,8,1,True,True,True,False,True
98,2015-01-01 08:36:00,IEDINBUR6,9.9,1.8,0.5,82.0,15.8,2015-01-01,08:36:00,3,8,1,True,True,True,False,True
99,2015-01-01 08:41:00,IEDINBUR6,9.9,1.8,0.5,83.0,6.4,2015-01-01,08:41:00,3,8,1,True,True,True,False,True
113,2015-01-01 09:53:00,IEDINBUR6,10.3,1.8,1.0,79.0,11.6,2015-01-01,09:53:00,3,9,1,True,True,True,False,True


# Data summarisation and aggregation

With the data cleansed, we now have non-uniform samples of the weather at a given station throughout the year, at a sub-hour level. To make meaningful plots on this data, we can aggregate over the days and months to gain an overall view and to compare across stations.

In [53]:
# Looking at the working days only and create a daily data set of working days:
wet_cycling = data[data['working_day'] == True].groupby('day')['get_wet_cycling'].any()
wet_cycling = pd.DataFrame(wet_cycling).reset_index()

# Group by month for display - monthly data set for plots.
wet_cycling['month'] = wet_cycling['day'].apply(lambda x: x.month)
monthly = wet_cycling.groupby('month')['get_wet_cycling'].value_counts().reset_index(drop=True)
monthly.rename(columns={"get_wet_cycling":"Rainy", 0:"Days"}, inplace=True)
monthly.replace({"Rainy": {True: "Wet", False:"Dry"}}, inplace=True)    
#monthly['month_name'] = monthly['month'].apply(lambda x: calendar.month_abbr[x])
#monthly.head()

In [42]:

# Get aggregate stats for each day in the dataset on rain in general - for heatmaps.
rainy_days = data.groupby(['day']).agg({
        "rain": {"rain": lambda x: (x > 0.0).any(),
                 "rain_amount": "sum"},
        "total_rain": {"total_rain": "max"},
        "get_wet_cycling": {"get_wet_cycling": "any"}
        })   

In [43]:
# clean up the aggregated data to a more easily analysed set:
rainy_days.reset_index(drop=False, inplace=True) # remove the 'day' as the index
rainy_days.rename(columns={"":"date"}, inplace=True) # The old index column didn't have a name - add "date" as name
rainy_days.columns = rainy_days.columns.droplevel(level=0) # The aggregation left us with a multi-index
                                                           # Remove the top level of this index.
rainy_days['rain'] = rainy_days['rain'].astype(bool)   

In [60]:
# Add the number of rainy hours per day this to the rainy_days dataset.
temp = data.groupby(["day", "hour_of_day"])['raining'].any()
temp = temp.groupby(level=[0]).sum().reset_index()
temp.rename(columns={'raining': 'hours_raining'}, inplace=True)
#temp['day'] = temp['day'].apply(lambda x: x.to_datetime().date())
rainy_days = rainy_days.merge(temp, left_on='date', right_on='day', how='left')
rainy_days = rainy_days.drop('day', axis=1)
rainy_days.head()
#temp.head()

,date,rain,rain_amount,total_rain,get_wet_cycling,hours_raining_x,hours_raining_y,hours_raining_x,hours_raining_y,hours_raining_x,hours_raining_y,hours_raining
0,2015-01-01,True,52.2,4.6,True,10.0,10.0,10.0,10.0,10.0,10.0,10.0
1,2015-01-02,True,52.0,4.8,True,8.0,8.0,8.0,8.0,8.0,8.0,8.0
2,2015-01-03,False,0.0,0.0,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2015-01-04,False,0.0,0.0,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2015-01-05,False,0.0,0.0,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [52]:
print("In the year, there were {} rainy days of {} at {}".format(rainy_days['rain'].sum(), len(rainy_days), station))    
print("It was wet while cycling {} working days of {} at {}".format(wet_cycling['get_wet_cycling'].sum(), 
                                                      len(wet_cycling),
                                                     station))
print("You get wet cycling {} % of the time!!".format(wet_cycling['get_wet_cycling'].sum()*1.0*100/len(wet_cycling)))


In the year, there were 194 rainy days of 364 at IEDINBUR6
It was wet while cycling 65 working days of 260 at IEDINBUR6
You get wet cycling 25.0 % of the time!!


# Visualisation using Pandas and Seaborn

# AGGREGATION WITH PHONE DATA

In [5]:
from preprocessor import *
import dateutil
# Load data from csv file
data = pd.read_csv('phone_data.csv',index_col=0)
prep = Preprocessor()
prep.overview(data)


Dataset contains 830 rows and 6 columns

Data types of the raw (uncleaned) data:
date             object
duration        float64
item             object
month            object
network          object
network_type     object
dtype: object

Categorical features of the dataset: ['date', 'item', 'month', 'network', 'network_type']

Numeric features of the dataset: ['duration']

Checking for features with null values...

network_type    0
network         0
month           0
item            0
duration        0
date            0
dtype: int64


In [6]:
 # Convert date from string to date times
    data['date'] = data['date'].apply(dateutil.parser.parse, dayfirst=True)
    data.head()

,date,duration,item,month,network,network_type
index,,,,,,
0,2014-10-15 06:58:00,34.429,data,2014-11,data,data
1,2014-10-15 06:58:00,13.000,call,2014-11,Vodafone,mobile
2,2014-10-15 14:46:00,23.000,call,2014-11,Meteor,mobile
3,2014-10-15 14:48:00,4.000,call,2014-11,Tesco,mobile
4,2014-10-15 17:27:00,4.000,call,2014-11,Tesco,mobile


__Summarising the DataFrame__



Once the data has been loaded into Python, Pandas makes the calculation of different statistics very simple. For example, mean, max, min, standard deviations and more for columns are easily calculable:

In [10]:
# What was the longest phone call / data entry?
duration = data['duration'].max()
print(f'Longest phone call /data entry: {duration}\n')

# How many seconds of phone calls are recorded in total?
phone_calls = data['duration'][data['item'] == 'call'].sum()
print(f'Total seconds of phone calls recorded: { phone_calls }\n')


# How many entries are there for each month?
entries = data['month'].value_counts()
print(f'Entries for each month: {entries}\n')
      

# Number of non-null unique network entries
unique_entries = data['network'].nunique()
print(f'Number of unique network entries: {unique_entries}\n')


Longest phone call /data entry: 10528.0

Total seconds of phone calls recorded: 92321.0

Entries for each month: 2014-11    230
2015-01    205
2014-12    157
2015-02    137
2015-03    101
Name: month, dtype: int64

Number of unique network entries: 9



__Summarising Groups in the DataFrame__

The groupby() function returns a GroupBy object, but essentially describes how the rows of the original data set has been split. the GroupBy object .groups variable is a dictionary whose keys are the computed unique groups and corresponding values being the axis labels belonging to each group. For example:

In [12]:
data.groupby(['month']).groups.keys()

dict_keys(['2014-11', '2014-12', '2015-01', '2015-02', '2015-03'])

In [11]:
len(data.groupby(['month']).groups['2014-11'])

230

Functions like __max(), min(), mean(), first(), last()__ can be quickly applied to the __GroupBy__ object to obtain summary statistics for each group – an immensely useful function.

In [13]:
# Get the first entry for each month
data.groupby('month').first()

,date,duration,item,network,network_type
month,,,,,
2014-11,2014-10-15 06:58:00,34.429,data,data,data
2014-12,2014-11-13 06:58:00,34.429,data,data,data
2015-01,2014-12-13 06:58:00,34.429,data,data,data
2015-02,2015-01-13 06:58:00,34.429,data,data,data
2015-03,2015-02-12 20:15:00,69.000,call,landline,landline


In [14]:
# Get the sum of the durations per month
data.groupby('month')['duration'].sum()

month
2014-11    26639.441
2014-12    14641.870
2015-01    18223.299
2015-02    15522.299
2015-03    22750.441
Name: duration, dtype: float64

In [15]:
# Get the number of dates / entries in each month
data.groupby('month')['date'].count()

month
2014-11    230
2014-12    157
2015-01    205
2015-02    137
2015-03    101
Name: date, dtype: int64

In [16]:
# What is the sum of durations, for calls only, to each network
data[data['item'] == 'call'].groupby('network')['duration'].sum()

network
Meteor        7200.0
Tesco        13828.0
Three        36464.0
Vodafone     14621.0
landline     18433.0
voicemail     1775.0
Name: duration, dtype: float64

You can also group by more than one variable, allowing more complex queries.

In [17]:
# How many calls, sms, and data entries are in each month?
data.groupby(['month', 'item'])['date'].count()

month    item
2014-11  call    107
         data     29
         sms      94
2014-12  call     79
         data     30
         sms      48
2015-01  call     88
         data     31
         sms      86
2015-02  call     67
         data     31
         sms      39
2015-03  call     47
         data     29
         sms      25
Name: date, dtype: int64

In [18]:
# How many calls, texts, and data are sent per month, split by network_type?
data.groupby(['month', 'network_type'])['date'].count()

month    network_type
2014-11  data             29
         landline          5
         mobile          189
         special           1
         voicemail         6
2014-12  data             30
         landline          7
         mobile          108
         voicemail         8
         world             4
2015-01  data             31
         landline         11
         mobile          160
         voicemail         3
2015-02  data             31
         landline          8
         mobile           90
         special           2
         voicemail         6
2015-03  data             29
         landline         11
         mobile           54
         voicemail         4
         world             3
Name: date, dtype: int64

The output from a groupby and aggregation operation varies between Pandas Series and Pandas Dataframes, which can be confusing for new users. As a rule of thumb, if you calculate more than one column of results, your result will be a Dataframe. For a single column of results, the agg function, by default, will produce a Series.